In [ ]:
def generate_round_keys(master_key, num_rounds):
    """
    Generate round keys from a master key
    Simple implementation: each round key is a rotated version of the master key
    """
    round_keys = []
    key = master_key
    key_length = len(master_key)

    for i in range(num_rounds):
        # Rotate the key left by i+1 positions for each round
        rotation = (i + 1) % key_length
        key = key[rotation:] + key[:rotation]
        round_keys.append(key)

    return round_keys

def round_function(right_half, round_key):
    """
    Round function that mixes the right half with the round key
    This is a simple implementation using XOR and bit rotation
    """
    # Ensure both inputs are the same length
    min_length = min(len(right_half), len(round_key))
    right_half = right_half[:min_length]
    round_key = round_key[:min_length]

    # XOR the right half with the round key
    result = xor(right_half, round_key)

    # Perform a simple substitution (rotate bits left by 1)
    result = result[1:] + result[0]

    return result

def xor(data1, data2):
    """Perform XOR operation on two binary strings"""
    result = ""
    for bit1, bit2 in zip(data1, data2):
        result += "1" if bit1 != bit2 else "0"
    return result

def feistel_cipher(plaintext, keys, round_function, num_rounds=16):
    """
    Implement a Feistel cipher

    Args:
        plaintext: Binary input data to encrypt
        keys: List of round keys
        round_function: Function to use in each round
        num_rounds: Number of rounds to apply

    Returns:
        Encrypted ciphertext
    """
    # Ensure plaintext length is even by padding if necessary
    if len(plaintext) % 2 == 1:
        plaintext += "0"

    # Split plaintext into left and right halves
    half_length = len(plaintext) // 2
    left = plaintext[:half_length]
    right = plaintext[half_length:]

    print(f"Initial state: L0={left}, R0={right}")

    # Apply the specified number of rounds
    for i in range(num_rounds):
        # Calculate round function output
        f_output = round_function(right, keys[i])

        # XOR left half with round function output
        new_right = xor(left, f_output)

        print(f"Round {i+1}: L{i+1}={right}, R{i+1}={new_right}, Key={keys[i]}, F()={f_output}")

        # Swap halves for next round
        left = right
        right = new_right

    # For the final result, don't swap the halves - concatenate them
    # If num_rounds is odd, swap them one last time
    if num_rounds % 2 == 1:
        ciphertext = right + left
    else:
        ciphertext = left + right

    return ciphertext

def decrypt_feistel(ciphertext, keys, round_function, num_rounds=16):
    """
    Decrypt using Feistel cipher by reversing the key order
    """
    return feistel_cipher(ciphertext, keys[::-1], round_function, num_rounds)

def string_to_binary(text):
    """Convert a string to binary representation"""
    binary = ""
    for char in text:
        # Convert each character to 8-bit binary
        binary += format(ord(char), '08b')
    return binary

def binary_to_string(binary):
    """Convert binary back to a string"""
    text = ""
    # Process 8 bits at a time
    for i in range(0, len(binary), 8):
        byte = binary[i:i+8]
        if len(byte) == 8:  # Ensure we have a full byte
            text += chr(int(byte, 2))
    return text

# Example usage
def run_example():
    # Set up parameters
    plaintext = "HELLO"
    master_key = "1010101010101010"  # 16-bit key for simplicity
    num_rounds = 4

    # Convert plaintext to binary
    binary_plaintext = string_to_binary(plaintext)
    print(f"Plaintext: {plaintext}")
    print(f"Binary plaintext: {binary_plaintext}")

    # Generate round keys
    round_keys = generate_round_keys(master_key, num_rounds)
    print(f"Master key: {master_key}")
    print(f"Round keys: {round_keys}")

    # Encrypt
    print("\n=== ENCRYPTION ===")
    ciphertext = feistel_cipher(binary_plaintext, round_keys, round_function, num_rounds)
    print(f"\nFinal ciphertext (binary): {ciphertext}")

    # Convert binary ciphertext to text representation (may not be readable)
    text_ciphertext = binary_to_string(ciphertext)
    print(f"Ciphertext as text: {text_ciphertext}")

    # Decrypt
    print("\n=== DECRYPTION ===")
    decrypted_binary = decrypt_feistel(ciphertext, round_keys, round_function, num_rounds)
    print(f"\nDecrypted (binary): {decrypted_binary}")

    # Convert binary back to text
    decrypted_text = binary_to_string(decrypted_binary)
    print(f"Decrypted text: {decrypted_text}")

    # Verifyk
    print(f"\nOriginal plaintext matches decrypted text: {plaintext == decrypted_text}")

# Run the example
run_example()


Plaintext: HELLO
Binary plaintext: 0100100001000101010011000100110001001111
Master key: 1010101010101010
Round keys: ['0101010101010101', '0101010101010101', '1010101010101010', '1010101010101010']

=== ENCRYPTION ===
Initial state: L0=01001000010001010100, R0=11000100110001001111
Round 1: L1=11000100110001001111, R1=0110101101100110, Key=0101010101010101, F()=0010001100100011
Round 2: L2=0110101101100110, R2=1011100010100010, Key=0101010101010101, F()=0111110001100110
Round 3: L3=1011100010100010, R3=0100111101110110, Key=1010101010101010, F()=0010010000010000
Round 4: L4=0100111101110110, R4=0111001100011011, Key=1010101010101010, F()=1100101110111001

Final ciphertext (binary): 01001111011101100111001100011011
Ciphertext as text: Ovs

=== DECRYPTION ===
Initial state: L0=0100111101110110, R0=0111001100011011
Round 1: L1=0111001100011011, R1=1111110000010101, Key=1010101010101010, F()=1011001101100011
Round 2: L2=1111110000010101, R2=1101111001100101, Key=1010101010101010, F()=10101